This [Jupyter notebook](https://jupyter.org/)
is meant to be viewed in "slide mode".

If you want to view the slides locally,
you'll need to install [RISE](https://rise.readthedocs.io/).

But it's probably simpler to just visit the
[online version](https://nbviewer.jupyter.org/format/slides/github/mgeier/splines2021/blob/main/presentation.ipynb).

# Rotation Splines for Spatial Audio Scenes

Matthias Geier

https://github.com/mgeier/splines2021

## Overview

* ...
  * ...
* ...
* ...

## The Audio Scene Description Format (ASDF)

* previously, only static positions, now sources can move along trajectories
* https://AudioSceneDescriptionFormat.readthedocs.io/
* implemented in Rust: https://github.com/AudioSceneDescriptionFormat/asdf-rust
* heavy use of splines: https://splines.readthedocs.io/
* not yet in the SSR, but there is a *pull request*: https://github.com/SoundScapeRenderer/ssr/pull/155
* ...